In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image

In [15]:
def readImagesAndTimes():
    filenames = ["img_0.033.jpg", "img_0.25.jpg", "img_2.5.jpg", "img_15.jpg"]
    times = np.array([1/30.0, 0.25, 2.5, 15.0], dtype=np.float32)
    
    images = []
    for filename in filenames:
        im = cv2.imread(filename)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        images.append(im)
        
    return images, times

In [16]:
images, times = readImagesAndTimes()

# Align Images
alignMTB = cv2.createAlignMTB()
alignMTB.process(images, images)

In [18]:
# What the Estimate Camera Response Function
calibrateDebevec = cv2.createCalibrateDebevec()
responseDebevec = calibrateDebevec.process(images, times)
y = np.squeeze(responseDebevec)

print(y)
print(type(y))

[[3.00185895e-03 3.14569124e-03 3.17693409e-03]
 [5.13659883e-03 5.50995301e-03 5.12363156e-03]
 [8.47017951e-03 9.15919896e-03 7.96243642e-03]
 [1.30432099e-02 1.37314368e-02 1.13811744e-02]
 [1.88275985e-02 1.87253579e-02 1.52361132e-02]
 [2.55967863e-02 2.36767102e-02 1.97298490e-02]
 [3.28335837e-02 2.83965077e-02 2.49761622e-02]
 [3.99508402e-02 3.31294648e-02 3.07730399e-02]
 [4.64823656e-02 3.81414481e-02 3.68419141e-02]
 [5.22717237e-02 4.35576104e-02 4.31310460e-02]
 [5.72223477e-02 4.93757799e-02 4.93940748e-02]
 [6.12765290e-02 5.55330776e-02 5.53188697e-02]
 [6.45324215e-02 6.20605238e-02 6.10328615e-02]
 [6.69973716e-02 6.87017441e-02 6.69421256e-02]
 [6.90519661e-02 7.48534575e-02 7.30631426e-02]
 [7.10246190e-02 8.04266706e-02 7.92777240e-02]
 [7.31588006e-02 8.54382366e-02 8.56516883e-02]
 [7.54613057e-02 8.97882879e-02 9.21954066e-02]
 [7.79159442e-02 9.34764668e-02 9.88777652e-02]
 [8.07081908e-02 9.68082696e-02 1.05987497e-01]
 [8.37928206e-02 1.00051053e-01 1.135570